# Merge ESPN and FantasyPros

In [1]:
from pathlib import Path

import pandas as pd
from rapidfuzz import fuzz

pd.set_option('max_rows', 100)

In [2]:
DATA_DIR = Path.cwd() / 'data'

# manual map created bc they are the most different
FP_TO_ESPN = {
    'Tampa Bay Buccaneers': 'Buccaneers D/ST',
    'Pittsburgh Steelers': 'Steelers D/ST',
    'Baltimore Ravens': 'Ravens D/ST',
    'Washington Football Team': 'Washington D/ST',
    'San Francisco 49ers': '49ers D/ST',
    'Buffalo Bills': 'Bills D/ST',
    'Denver Broncos': 'Broncos D/ST',
    'New England Patriots': 'Patriots D/ST',
    'Los Angeles Rams': 'Rams D/ST',
    'Cleveland Browns': 'Browns D/ST',
    'Indianapolis Colts': 'Colts D/ST',
    'Minnesota Vikings': 'Vikings D/ST',
    'Miami Dolphins': 'Dolphins D/ST',
    'New Orleans Saints': 'Saints D/ST',
    'Green Bay Packers': 'Packers D/ST',
    'Chicago Bears': 'Bears D/ST',
    'New York Giants': 'Giants D/ST',
    'Arizona Cardinals': 'Cardinals D/ST',
    'New York Jets': 'Jets D/ST',
    'Kansas City Chiefs': 'Chiefs D/ST',
    'Tennessee Titans': 'Titans D/ST',
    'Atlanta Falcons': 'Falcons D/ST',
    'Carolina Panthers': 'Panthers D/ST',
    'Seattle Seahawks': 'Seahawks D/ST',
    'Los Angeles Chargers': 'Chargers D/ST',
    'Philadelphia Eagles': 'Eagles D/ST',
    'Cincinnati Bengals': 'Bengals D/ST',
    'Las Vegas Raiders': 'Raiders D/ST',
    'Dallas Cowboys': 'Cowboys D/ST',
    'Jacksonville Jaguars': 'Jaguars D/ST',
    'Detroit Lions': 'Lions D/ST',
    'Houston Texans': 'Texans D/ST',

    # extras
    'Phillip Walker': 'P.J. Walker',
    'Patrick Mahomes II': 'Patrick Mahomes',
    'D.K. Metcalf': 'DK Metcalf',
    'D.J. Moore': 'DJ Moore',
    'Darrell Henderson': 'Darrell Henderson Jr.',
    'Will Fuller V': 'William Fuller V',
    'D.J. Chark Jr.': 'DJ Chark Jr.',
    'Chris Herndon IV': 'Chris Herndon',
    'Phillip Walker': 'Philip Rivers',
    'Keelan Cole Sr.': 'Keelan Cole',
    "D'Wayne Eskridge": 'Dee Eskridge',
    'Josh Palmer': 'Joshua Palmer',
    'Mohamed Sanu': 'Mohamed Sanu Sr.',
}

SKIP = set(['Jose Borregales', 'Chris Naggar', 'Andrew Jacas', 'Riley Patterson', 'Sammis Reyes',
            'Tanner Hudson', 'Bryce Perkins', 'Drew Stanton'])

In [3]:
def top_match(item: str, search_list: list[str]) -> str:
    ans, max_ratio = '', 0
    for s in search_list:
            r = fuzz.ratio(item, s, score_cutoff=0.5)
            if r > max_ratio:
                max_ratio = r
                ans = s

    if max_ratio < 0.5:
        return None
    return ans

In [4]:
fp = pd.read_csv(DATA_DIR / 'fantasypros_2021_draft_rankings_all.csv')
fp = fp[~fp['name'].isin(SKIP)].copy()

# change some FP names to match ESPN names
for fp_name, espn_name in FP_TO_ESPN.items():
    fp.loc[fp['name'] == fp_name, 'name'] = espn_name

espn = pd.read_csv(DATA_DIR / 'espn_draft_trends_2021.csv')
espn = espn[~espn['name'].isin(SKIP)].copy()

In [5]:
merged = pd.merge(fp, espn, on=['name', 'pos'], how='left')

espn_pos_to_names_dict = espn.groupby('pos')['name'].apply(lambda v: list(v)).to_dict()
missing_names = merged[merged['player_id'].isnull()][['name', 'pos']].set_index('name').to_dict('dict').get('pos')

len(missing_names)

29

In [6]:
# good enough
best_match = {}
for name, pos in missing_names.items():
    if pos != 'DST':
        espn_names = espn_pos_to_names_dict.get(pos)
        best_match[name] = top_match(name, espn_names)

best_match

{'Tommy Hudson': 'Tommy Sweeney',
 'Cordarrelle Patterson': 'Andre Patton',
 'J.P. Holtz': 'Jacob Hollister',
 'Nick Bowers': 'Nick Boyle',
 'Ryan Becker': 'Nate Becker',
 'Mitchell Wilcox': 'Will Dissly',
 'Ty Montgomery': 'David Montgomery',
 'Demetric Felton': 'Darius Slayton',
 'Eli Wolf': 'Clive Walford',
 'Jake Breeland': 'Blake Bell',
 'Andrew Beck': 'Nate Becker',
 'Giovanni Ricci': 'Jonnu Smith',
 'Jack Stoll': 'Jack Doyle',
 'Mekhi Sargent': 'Chris Warren',
 'Tre Harbison': 'Tra Carson',
 'K.J. Hill Jr.': 'KJ Hill Jr.',
 'James Proche': 'James Proche II',
 'Josh Hokit': 'Josh Jacobs',
 'Hakeem Butler': 'Emmanuel Butler',
 'Chris Finke': 'Chris Godwin',
 'Victor Bolden Jr.': 'Victor Bolden',
 'Stanley Morgan Jr.': 'Stanley Morgan',
 'Juwan Johnson': 'Cade Johnson',
 'T.J. Vasher': 'C.J. Board',
 'Austin Trammell': 'Austin Proehl',
 'Tyler Simmons': 'Tyler Johnson',
 'Rysen John': 'Tyler Johnson',
 'Stephen Guidry': 'Steven Sims',
 'Warren Jackson': 'DeSean Jackson'}

## Export cheatsheet

In [7]:
for col in merged.columns[7:]:
    merged[col] = merged[col].astype('Int64')  # nullable int

merged.head(10)

,tiers,pos,name,bye,rk,team,sos,ecr_vs_adp,player_id,ppr_rank,avg_draft_pos,pos_diff,ppr_auc_value,avg_auc_value,auc_diff
0,1,WR,Davante Adams,13,1,GB,1,0,16800,7,7,0,51,56,5
1,1,WR,Tyreek Hill,12,2,KC,3,0,3116406,13,11,-2,44,51,7
2,1,WR,Stefon Diggs,7,3,BUF,3,0,2976212,16,16,0,40,45,5
3,1,TE,Mark Andrews,8,4,BAL,4,1,3116365,63,48,-15,9,11,2
4,1,TE,T.J. Hockenson,9,5,DET,4,1,4036133,64,62,-2,8,8,0
5,1,TE,Travis Kelce,12,1,KC,3,0,15847,6,8,2,53,55,2
6,1,TE,Darren Waller,8,2,LV,4,0,2576925,24,25,1,32,32,0
7,1,TE,George Kittle,6,3,SF,4,0,3040151,29,27,-2,28,30,2
8,1,TE,Kyle Pitts,6,6,ATL,2,-2,4360248,65,52,-13,8,13,5
9,1,RB,Alvin Kamara,6,3,NO,2,1,3054850,4,4,0,58,65,7


In [8]:
merged.to_csv(DATA_DIR / 'cheatsheet_2021.csv', index=False)